# Data Import, Cleaning, and Preparation

This module is used to query the postgreSQL database in order to retrieve the Yelp and Violations dataset. There will be several steps to do this:
1. Import flattened violations dataset. We will use a SQL query to perform aggregation of violation data by restaurant and inspeciton date. 
2. Import Yelp business data and join to inspection data    
4. Join the Yelp business and Inspection Data with the Yelp Review Data
    + Reviews for a given establishment will be aggregated so that reviews *after* the previous inspection (or the earliest review date) and *before* the date of a given inspection are in one batch. 
    +  Aggregate any review "count" features using this same logic
    + Combine the review documents for a restaurant into a CLOB using the same logic
    
### TO DO:
1. Determine how we want to do Levenshtein matching to combine the datasets (see the *Join Yelp Review Data with Inspection Dataset* below) 
2. Create additional engineered features
3. n-gram extraction
4. vectorization of n-grams
5. model selection
    + training
    + validaiton
    + evaluation
    +repeat with additional or removed features and data segmentation (if aggregating the review text does not yield successful results)

## Import and Clean Data

In [1]:
import psycopg2 as psy
import pandas as pd
import re
import numpy as np
import ast

In [2]:
#set up connection to our DB
conn = psy.connect(database="sterndsyelp", 
                        user="mvsternds", 
                        password="nyustern123!", 
                        host="sterndsyelp.cawzspvmqd5q.us-east-1.rds.amazonaws.com", 
                        port="5432"
                       )
#open cursor and check our tables in the DB
cur = conn.cursor()

In [3]:
#cur.execute("SELECT * FROM public.restaurants LIMIT 50 ")
#biz = pd.DataFrame(cur.fetchall())

cur.execute("SELECT * FROM public.toronto_checkins LIMIT 50")
checkins = pd.DataFrame(cur.fetchall())

cur.execute("SELECT * FROM public.toronto_reviews LIMIT 50")
reviews = pd.DataFrame(cur.fetchall())

**NOTE: ONLY LIMITING to 50 rows during build phase to limit processing time. **

In [4]:
reviews.columns = ['bizID','reviewID','userID','type','stars','text','useful','funny','cool','date']
#get total reviews per biz
rev = reviews['bizID'].value_counts()
rev_counts = pd.DataFrame(rev).reset_index()
rev_counts.columns = ['bizID','all_review_count']

In [5]:
#not using this - can delete

checkins.columns = ['bizID','type','datetime']
#get total checkins per biz
chks  = checkins['bizID'].value_counts()
chk_counts = pd.DataFrame(chks).reset_index()
chk_counts.columns = ['bizID','checkin_counts']

### Join Yelp Review Data with Inspection Dataset

We have a few options here. While it is optimal to have as much done in Python as possible, the matching process in python is impractically slow. We can 1) use a manual implementation of Levenshtein Distance (LD), 2) use a package with optimized LD code, or 3) do the joining of the inspeciton data and review data in our database. OPtions 1& 2 are shown below, and code to retrieve the results of option 3 are at the bottom of this section.

If we decide to go with option 3, the last steps are to combine all records where the business ID, last inspection date, and inspection date are equal in order to get to one observation per restaurant-inspection combination. We can then add in any other engineered features.

**note: next cell should return matches once we include more than the 50 rows (fingers crossed)**

#### Levenshtein Option #3 (in-database) 
This option joins the yelp restaurant informaiton to each inspection record where:
 * The Levenshetein distance of the restaurant name from the two datasets is <3
 * The distance of the address from each dataset is <4
 * The date of the review is greater than the prior inspection date
 * The date of the review is less than or equal to inspeciton date on the record
 
Whitespace at the beginning and end of the name and address in each dataset is trimmed, and the strings are converted to uppercase before matching. The mathcing thresholds can be adjusted to increase potential for matching, or decrease false matches.

In [6]:
# The materialized view of the restaurant, inspection, and review data is "toronto_all"
cur.execute("SELECT * FROM public.toronto_all where review_date is not null LIMIT 50" )
obs = pd.DataFrame(cur.fetchall())
obs.head()
obs.columns=['bizID','name','address','postal_code','neighborhood','lat','long','categories','attributes','is_open','review_cnt','hours','stars','setablishment_id','establishment_name','establishment_address','inspection_date','last_inspection','count_minor','count_sig','count_crucial','count_na','count_crucial_signficant','review_id','user_id','review_stars','review_text','useful','funny','cool','review_dt']
obs.head()

,bizID,name,address,postal_code,neighborhood,lat,long,categories,attributes,is_open,...,count_na,count_crucial_signficant,review_id,user_id,review_stars,review_text,useful,funny,cool,review_dt
0,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,0,0,OriPsMQx1cRyE9hgNnhQFg,m7z-tX6XDZ27xGhGjnI21w,5,"""I am confident when I say this place is hands...",4,0,4,2016-02-15
1,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,0,0,1WON5dUarKdWJ6-yq7Ne6g,UrfdzamoBt0WW9Ifqy7RIw,5,"The short version of my review is this: ""Hashi...",3,0,1,2015-11-18
2,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,0,0,TNpm4Hs6x2yJvYbUZSeA4w,5KbUkX5DHGtDSmqdG5LLhw,5,"""It's a great cultural experience with great f...",0,0,0,2016-03-14
3,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,0,0,0pS7e898Z5AywcZQ9_DX1Q,xspGyCnzmZgsP-VUnQ3K4A,5,"""This was definitely where quality of food and...",1,0,1,2016-03-15
4,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,0,0,ePOLGcC5yJbD_DzOLfyKbA,_dRcIdWjks0phgfiY27AFQ,5,My sister and I enjoyed the lunchtime Kaiseki ...,0,0,0,2016-05-01


In [7]:
obs['bizID-dt'] = obs['bizID'] + "-" + obs['inspection_date'].map(str)

In [8]:
in_scope_rev = obs['bizID-dt'].value_counts()
in_scope_reviews = pd.DataFrame(in_scope_rev).reset_index()
in_scope_reviews.columns = ['bizID-dt','count_reviews_in_scope']
in_scope_reviews.head()

,bizID-dt,count_reviews_in_scope
0,02BXFKzu1rgaYulNGYvi6g-2015-10-08,20
1,02BXFKzu1rgaYulNGYvi6g-2016-01-19,15
2,02BXFKzu1rgaYulNGYvi6g-2016-12-28,7
3,01l8MH9tBK6GPvJQdMU1gw-2016-03-22,4
4,01nRNgH_ukm8E2td9TTZDA-2016-10-21,2


In [9]:
#get dummies for star rating column
obs = pd.concat([obs, pd.get_dummies(obs['review_stars'], prefix='stars')], axis=1)
obs.head()

,bizID,name,address,postal_code,neighborhood,lat,long,categories,attributes,is_open,...,useful,funny,cool,review_dt,bizID-dt,stars_1,stars_2,stars_3,stars_4,stars_5
0,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,4,0,4,2016-02-15,01l8MH9tBK6GPvJQdMU1gw-2016-03-22,0,0,0,0,1
1,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,3,0,1,2015-11-18,01l8MH9tBK6GPvJQdMU1gw-2016-03-22,0,0,0,0,1
2,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,0,0,0,2016-03-14,01l8MH9tBK6GPvJQdMU1gw-2016-03-22,0,0,0,0,1
3,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,1,0,1,2016-03-15,01l8MH9tBK6GPvJQdMU1gw-2016-03-22,0,0,0,0,1
4,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,0,0,0,2016-05-01,01l8MH9tBK6GPvJQdMU1gw-2016-11-17,0,0,0,0,1


In [10]:
stars = obs.groupby('bizID-dt')[['stars_1', 'stars_2','stars_3','stars_4','stars_5']].sum().reset_index()
stars.head()

,bizID-dt,stars_1,stars_2,stars_3,stars_4,stars_5
0,01l8MH9tBK6GPvJQdMU1gw-2016-03-22,0,0,0,0,4
1,01l8MH9tBK6GPvJQdMU1gw-2016-11-17,0,0,0,0,1
2,01nRNgH_ukm8E2td9TTZDA-2016-05-09,1,0,0,0,0
3,01nRNgH_ukm8E2td9TTZDA-2016-10-21,0,1,0,0,1
4,02BXFKzu1rgaYulNGYvi6g-2015-10-08,2,2,7,5,4


In [11]:
combined_revs = obs.groupby('bizID-dt')['review_text'].apply(' '.join).reset_index()
combined_revs.head()

,bizID-dt,review_text
0,01l8MH9tBK6GPvJQdMU1gw-2016-03-22,"""I am confident when I say this place is hands..."
1,01l8MH9tBK6GPvJQdMU1gw-2016-11-17,My sister and I enjoyed the lunchtime Kaiseki ...
2,01nRNgH_ukm8E2td9TTZDA-2016-05-09,"""The worst treatment in my entire life.\n\nThe..."
3,01nRNgH_ukm8E2td9TTZDA-2016-10-21,"""Stopped by while shopping in the eaton centre..."
4,02BXFKzu1rgaYulNGYvi6g-2015-10-08,"""I live 5 minutes away from this restaurant an..."


In [12]:
users = obs.groupby('bizID-dt')['user_id'].count().reset_index()
users.columns = ['bizID-dt','count_unique_users']
users.head()

,bizID-dt,count_unique_users
0,01l8MH9tBK6GPvJQdMU1gw-2016-03-22,4
1,01l8MH9tBK6GPvJQdMU1gw-2016-11-17,1
2,01nRNgH_ukm8E2td9TTZDA-2016-05-09,1
3,01nRNgH_ukm8E2td9TTZDA-2016-10-21,2
4,02BXFKzu1rgaYulNGYvi6g-2015-10-08,20


In [13]:
sub = obs[['bizID-dt','bizID','name','postal_code','lat','long','categories','attributes','is_open','count_crucial_signficant']]
sub = sub.drop_duplicates()
sub.head()

,bizID-dt,bizID,name,postal_code,lat,long,categories,attributes,is_open,count_crucial_signficant
0,01l8MH9tBK6GPvJQdMU1gw-2016-03-22,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,M3C 1Z5,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,0
4,01l8MH9tBK6GPvJQdMU1gw-2016-11-17,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,M3C 1Z5,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,0
5,01nRNgH_ukm8E2td9TTZDA-2016-05-09,01nRNgH_ukm8E2td9TTZDA,Baroli Cafe,M5B 2H1,43.6536106,-79.3800603,"['Coffee & Tea', 'Food']","['BikeParking: False', 'BusinessAcceptsCreditC...",1,3
6,01nRNgH_ukm8E2td9TTZDA-2016-10-21,01nRNgH_ukm8E2td9TTZDA,Baroli Cafe,M5B 2H1,43.6536106,-79.3800603,"['Coffee & Tea', 'Food']","['BikeParking: False', 'BusinessAcceptsCreditC...",1,0
8,02BXFKzu1rgaYulNGYvi6g-2015-10-08,02BXFKzu1rgaYulNGYvi6g,Matsuda Japanese Cuisine,M1W 2B4,43.831795518,-79.2663108185,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,0


In [14]:
#merge in all data into one df
df1 = pd.merge(sub,stars,on='bizID-dt', how='left')
df2 = pd.merge(df1,combined_revs,on='bizID-dt', how='left')
df3 = pd.merge(df2,rev_counts,on='bizID', how='left')
df4 = pd.merge(df3,in_scope_reviews,on='bizID-dt', how='left')
df5 = pd.merge(df4,users,on='bizID-dt', how='left')

In [66]:
t = []
for i in range(len(df5['categories'])):
    x = ast.literal_eval(df5['categories'][i])
    t.append(x)
    
cats = pd.DataFrame(t)
cats_df = pd.get_dummies(cats, prefix='Category')
cats_df = cats_df.groupby(cats_df.columns, axis=1).sum()


atts_df = pd.DataFrame()
for x in range(len(df5['attributes'])):
    list_yelp = ast.literal_eval(df5['attributes'][x])

    attribute_list = []
    attribute_name = []

    for i in list_yelp:

        name = i.split(":")[0]
        values = i.split( name+": " )[1].replace('{','').replace('}','')

        if len(values.split(":")) > 1: 
        
            for j in values.split(","):
                name_j = name + "_" + (j.split(":")[0].strip().replace("'",''))
                attribute_name.append( name_j )
                attribute_list.append (j.split(":")[1])
        else:
            attribute_name.append( name )
            attribute_list.append ( values )
        
    dataframe = pd.DataFrame(attribute_list).transpose()
    dataframe.columns = attribute_name
    atts_df = atts_df.append(dataframe)
    
atts_df = atts_df.reset_index().drop('index', 1)

df = pd.concat([df5, cats_df, atts_df], axis=1)

#make sure each bizID-dt is only appearing once in the data
print('Max of number of unique bizID-dt in df (should be 1):',max(df5['bizID-dt'].value_counts()))

df=pd.get_dummies(df, columns=['Category_Coffee & Tea',
 'Category_Food',
 'Category_Japanese',
 'Category_Restaurants',
 'Alcohol',
 'Ambience_casual',
 'Ambience_classy',
 'Ambience_hipster',
 'Ambience_intimate',
 'Ambience_romantic',
 'Ambience_touristy',
 'Ambience_trendy',
 'Ambience_upscale',
 'BikeParking',
 'BusinessAcceptsCreditCards',
 'BusinessParking_garage',
 'BusinessParking_lot',
 'BusinessParking_street',
 'BusinessParking_valet',
 'BusinessParking_validated',
 'Caters',
 'GoodForKids',
 'GoodForMeal_breakfast',
 'GoodForMeal_brunch',
 'GoodForMeal_dessert',
 'GoodForMeal_dinner',
 'GoodForMeal_latenight',
 'GoodForMeal_lunch',
 'HasTV',
 'NoiseLevel',
 'OutdoorSeating',
 'RestaurantsAttire',
 'RestaurantsDelivery',
 'RestaurantsGoodForGroups',
 'RestaurantsPriceRange2',
 'RestaurantsReservations',
 'RestaurantsTableService',
 'RestaurantsTakeOut',
 'WiFi', 'is_open','postal_code'], dummy_na=True)
df


Max of number of unique bizID-dt in df (should be 1): 1


,bizID-dt,bizID,name,lat,long,categories,attributes,count_crucial_signficant,stars_1,stars_2,stars_3,stars_4,stars_5,review_text,all_review_count,count_reviews_in_scope,count_unique_users,Category_Coffee & Tea_0.0,Category_Coffee & Tea_1.0,Category_Coffee & Tea_nan,Category_Food_0.0,Category_Food_1.0,Category_Food_nan,Category_Japanese_0.0,Category_Japanese_1.0,Category_Japanese_nan,Category_Restaurants_0.0,Category_Restaurants_1.0,Category_Restaurants_nan,Alcohol_beer_and_wine,Alcohol_nan,Ambience_casual_ False,Ambience_casual_ True,Ambience_casual_nan,Ambience_classy_ False,Ambience_classy_nan,Ambience_hipster_ False,Ambience_hipster_nan,Ambience_intimate_ False,Ambience_intimate_nan,Ambience_romantic_ False,Ambience_romantic_nan,Ambience_touristy_ False,Ambience_touristy_nan,Ambience_trendy_ False,Ambience_trendy_nan,Ambience_upscale_ False,Ambience_upscale_nan,BikeParking_False,BikeParking_True,BikeParking_nan,BusinessAcceptsCreditCards_True,BusinessAcceptsCreditCards_nan,BusinessParking_garage_ False,BusinessParking_garage_nan,BusinessParking_lot_ False,BusinessParking_lot_ True,BusinessParking_lot_nan,BusinessParking_street_ False,BusinessParking_street_nan,BusinessParking_valet_ False,BusinessParking_valet_nan,BusinessParking_validated_ False,BusinessParking_validated_nan,Caters_False,Caters_True,Caters_nan,GoodForKids_False,GoodForKids_True,GoodForKids_nan,GoodForMeal_breakfast_ False,GoodForMeal_breakfast_nan,GoodForMeal_brunch_ False,GoodForMeal_brunch_nan,GoodForMeal_dessert_ False,GoodForMeal_dessert_nan,GoodForMeal_dinner_ False,GoodForMeal_dinner_ True,GoodForMeal_dinner_nan,GoodForMeal_latenight_ False,GoodForMeal_latenight_nan,GoodForMeal_lunch_ False,GoodForMeal_lunch_ True,GoodForMeal_lunch_nan,HasTV_False,HasTV_nan,NoiseLevel_average,NoiseLevel_quiet,NoiseLevel_nan,OutdoorSeating_False,OutdoorSeating_True,OutdoorSeating_nan,RestaurantsAttire_casual,RestaurantsAttire_dressy,RestaurantsAttire_nan,RestaurantsDelivery_False,RestaurantsDelivery_nan,RestaurantsGoodForGroups_False,RestaurantsGoodForGroups_True,RestaurantsGoodForGroups_nan,RestaurantsPriceRange2_1,RestaurantsPriceRange2_2,RestaurantsPriceRange2_4,RestaurantsPriceRange2_nan,RestaurantsReservations_True,RestaurantsReservations_nan,RestaurantsTableService_True,RestaurantsTableService_nan,RestaurantsTakeOut_False,RestaurantsTakeOut_True,RestaurantsTakeOut_nan,WiFi_no,WiFi_nan,is_open_1,is_open_nan,postal_code_M1W 2B4,postal_code_M3C 1Z5,postal_code_M5B 2H1,postal_code_nan
0,01l8MH9tBK6GPvJQdMU1gw-2016-03-22,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",0,0,0,0,0,4,"""I am confident when I say this place is hands...",NaN,4,4,1,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,1,0,1,0,1,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0
1,01l8MH9tBK6GPvJQdMU1gw-2016-11-17,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",0,0,0,0,0,1,My sister and I enjoyed the lunchtime Kaiseki ...,NaN,1,1,1,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,1,0,1,0,1,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0
2,01nRNgH_ukm8E2td9TTZDA-2016-05-09,01nRNgH_ukm8E2td9TTZDA,Baroli Cafe,43.6536106,-79.3800603,"['Coffee & Tea', 'Food']","['BikeParking: False', 'BusinessAcceptsCreditC...",3,1,0,0,0,0,"""The worst treatment in my entire life.\n\nThe...",NaN,1,1,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,1,1,0,0,0,0,1,0,1,0,0,1,0,1,1,0,0,0,1,0
3,01nRNgH_ukm8E2td9TTZDA-2016-10-21,01nRNgH_ukm8E2td9TTZDA,Baroli Cafe,43.6536106,-79.3800603,"['Coffee & Tea', 'Food']","['BikeP

In [33]:
#useful code to view all columns of df

pd.set_option('display.max_columns', None)

In [67]:
backup_final_df = df

del df['categories']
del df['attributes']
del df['name']
del df['bizID']
del df['bizID-dt']
del df['lat']
del df['long']
df['count_crucial_signficant']= df['count_crucial_signficant']>0


In [68]:
from nltk.corpus import stopwords
import nltk
import string
stop = stopwords.words('english')
df['review_text']=df['review_text'].str.lower()
df['review_text'] = df['review_text'].apply(lambda x: '  '.join([word for word in x.split() if word not in (stop)]))
ps = nltk.stem.WordNetLemmatizer()
df['review_text']=df["review_text"].apply(lambda x:[ps.lemmatize(y,pos='v') for y in x.split()])
df['review_text']=df['review_text'].apply(lambda x: ',  '.join(x))
translator = str.maketrans('', '', string.punctuation)
df['review_text'] = df['review_text'].str.translate(translator)

list(df)
df

,count_crucial_signficant,stars_1,stars_2,stars_3,stars_4,stars_5,review_text,all_review_count,count_reviews_in_scope,count_unique_users,Category_Coffee & Tea_0.0,Category_Coffee & Tea_1.0,Category_Coffee & Tea_nan,Category_Food_0.0,Category_Food_1.0,Category_Food_nan,Category_Japanese_0.0,Category_Japanese_1.0,Category_Japanese_nan,Category_Restaurants_0.0,Category_Restaurants_1.0,Category_Restaurants_nan,Alcohol_beer_and_wine,Alcohol_nan,Ambience_casual_ False,Ambience_casual_ True,Ambience_casual_nan,Ambience_classy_ False,Ambience_classy_nan,Ambience_hipster_ False,Ambience_hipster_nan,Ambience_intimate_ False,Ambience_intimate_nan,Ambience_romantic_ False,Ambience_romantic_nan,Ambience_touristy_ False,Ambience_touristy_nan,Ambience_trendy_ False,Ambience_trendy_nan,Ambience_upscale_ False,Ambience_upscale_nan,BikeParking_False,BikeParking_True,BikeParking_nan,BusinessAcceptsCreditCards_True,BusinessAcceptsCreditCards_nan,BusinessParking_garage_ False,BusinessParking_garage_nan,BusinessParking_lot_ False,BusinessParking_lot_ True,BusinessParking_lot_nan,BusinessParking_street_ False,BusinessParking_street_nan,BusinessParking_valet_ False,BusinessParking_valet_nan,BusinessParking_validated_ False,BusinessParking_validated_nan,Caters_False,Caters_True,Caters_nan,GoodForKids_False,GoodForKids_True,GoodForKids_nan,GoodForMeal_breakfast_ False,GoodForMeal_breakfast_nan,GoodForMeal_brunch_ False,GoodForMeal_brunch_nan,GoodForMeal_dessert_ False,GoodForMeal_dessert_nan,GoodForMeal_dinner_ False,GoodForMeal_dinner_ True,GoodForMeal_dinner_nan,GoodForMeal_latenight_ False,GoodForMeal_latenight_nan,GoodForMeal_lunch_ False,GoodForMeal_lunch_ True,GoodForMeal_lunch_nan,HasTV_False,HasTV_nan,NoiseLevel_average,NoiseLevel_quiet,NoiseLevel_nan,OutdoorSeating_False,OutdoorSeating_True,OutdoorSeating_nan,RestaurantsAttire_casual,RestaurantsAttire_dressy,RestaurantsAttire_nan,RestaurantsDelivery_False,RestaurantsDelivery_nan,RestaurantsGoodForGroups_False,RestaurantsGoodForGroups_True,RestaurantsGoodForGroups_nan,RestaurantsPriceRange2_1,RestaurantsPriceRange2_2,RestaurantsPriceRange2_4,RestaurantsPriceRange2_nan,RestaurantsReservations_True,RestaurantsReservations_nan,RestaurantsTableService_True,RestaurantsTableService_nan,RestaurantsTakeOut_False,RestaurantsTakeOut_True,RestaurantsTakeOut_nan,WiFi_no,WiFi_nan,is_open_1,is_open_nan,postal_code_M1W 2B4,postal_code_M3C 1Z5,postal_code_M5B 2H1,postal_code_nan
0,False,0,0,0,0,4,i confident say place hand authentic kai...,NaN,4,4,1,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,1,0,1,0,1,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0
1,False,0,0,0,0,1,sister enjoy lunchtime kaiseki meal last ...,NaN,1,1,1,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,1,0,1,0,1,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0
2,True,1,0,0,0,0,the worst treatment entire lifennthe girl...,NaN,1,1,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,1,1,0,0,0,0,1,0,1,0,0,1,0,1,1,0,0,0,1,0
3,False,0,1,0,0,1,stopped shop eaton centre sandwich displa...,NaN,2,2,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,1,1,0,0,0,0,1,0,1,0,0,1,0,1,1,0,0,0,1,0
4,False,2,2,7,5,4,i live 5 minutes away restaurant ive co...,NaN,20,20,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,0,0
5,False,1,2,7,4,1,subpar ayce lunch sushi experiencennfoodnt...,NaN,15,15,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,

### Model Time!

In [71]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import scipy.sparse
import sklearn.pipeline as pipe

tfidf = pipe.Pipeline([
    ('vectorizer', HashingVectorizer(ngram_range=(1,4), non_negative=True)),
    ('tfidf', TfidfTransformer()),
])
X_text = df['review_text']
X_ntext = df.drop(['review_text','count_crucial_signficant'], axis=1)
X_tfidf = tfidf.fit_transform(X_text)
X = scipy.sparse.hstack([X_tfidf, X_ntext])
Y = df['count_crucial_signficant']


In [ ]:
model = LogisticRegression()
model.fit(X_train_binary, Y_train)
print ("Area under the ROC curve on test data = %.3f" % metrics.roc_auc_score(model.predict(X_test_binary), Y_test))
fpr, tpr, thresholds = metrics.roc_curve(Y_test, model.predict_proba(X_test_binary)[:,1])

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(X_train)

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

model_v2 = LogisticRegression()
model_v2.fit(X_train_tfidf, Y_train)
print ("Area under the ROC curve on test data = %.3f" % metrics.roc_auc_score(model_v2.predict(X_test_counts), Y_test))
